In [2]:
import pandas as pd
import requests
import numpy as np
import boto3
from io import StringIO
import os

In [3]:
api_key = 'Replace with your API key'  
base_url = 'http://api.openweathermap.org/data/2.5/weather'

In [6]:
df = pd.read_csv('worldcities.csv')
country_counts = df['country'].value_counts() >= 100
con = country_counts[country_counts == True].index

In [7]:
df = df[df['country'].isin(con)]
df['country'].value_counts()

country
India            7031
United States    5324
Brazil           2937
Germany          1747
China            1663
                 ... 
Korea, South      104
Costa Rica        104
Bangladesh        104
Bolivia           103
Austria           103
Name: count, Length: 74, dtype: int64

In [8]:
df.head()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id
0,Tokyo,Tokyo,35.6897,139.6922,Japan,JP,JPN,Tōkyō,primary,37732000.0,1392685764
1,Jakarta,Jakarta,-6.1750,106.8275,Indonesia,ID,IDN,Jakarta,primary,33756000.0,1360771077
2,Delhi,Delhi,28.6100,77.2300,India,IN,IND,Delhi,admin,32226000.0,1356872604
3,Guangzhou,Guangzhou,23.1300,113.2600,China,CN,CHN,Guangdong,admin,26940000.0,1156237133
4,Mumbai,Mumbai,19.0761,72.8775,India,IN,IND,Mahārāshtra,admin,24973000.0,1356226629


In [12]:
df = df.groupby('country').head(6)

In [13]:
def fetch_weather(city, lat, lng):
    params = {
        'lat': lat,
        'lon': lng,
        'appid': api_key,
        'units': 'metric'
    }
    response = requests.get(base_url, params=params)
    if response.status_code == 200:
        data = response.json()
        if 'main' in data and 'weather' in data or 'sys' in data:
            return {
                'city': city,
                'temperature': data['main']['temp'],
                'weather': data['weather'][0]['description'],
                'lat' : data['coord']['lat'],
                'lng' : data['coord']['lon'],
                'temp_min' : data['main']['temp_min'],
                'temp_max' : data['main']['temp_max'],
                'sea_level' : data['main']['sea_level'],
                'humidity' : data['main']['humidity'],
                'current_time' : data['dt'],
                'sunrise' : data['sys']['sunrise'],
                'sunset' : data['sys']['sunset'],
                'timezone' : data['timezone']
            }
        else:
            print(f"Missing 'main' or 'weather' in response for city: {city}")
            return {'city': city, 'temperature': None, 'weather': None}
    else:
        print(f"Failed to fetch weather data for city: {city}, status code: {response.status_code}")
        return {'city': city, 'temperature': None, 'weather': None}

In [14]:
# Fetch weather data for each city
weather_data = []
for index, row in df.iterrows():
    weather = fetch_weather(row['city'], row['lat'], row['lng'])
    weather_data.append(weather)

weather_df = pd.DataFrame(weather_data)

In [15]:
weather_df.head()

,city,temperature,weather,lat,lng,temp_min,temp_max,sea_level,humidity,current_time,sunrise,sunset,timezone
0,Tokyo,29.82,broken clouds,35.6903,139.6930,27.56,30.36,1000,74,1723306179,1723319820,1723368962,32400
1,Jakarta,28.34,few clouds,-6.1750,106.8275,26.70,29.13,1011,69,1723306688,1723244492,1723287281,25200
2,Delhi,29.10,mist,28.6100,77.2300,29.10,29.10,1001,89,1723306688,1723249063,1723296916,19800
3,Guangzhou,30.92,overcast clouds,23.1300,113.2600,30.92,30.92,1003,65,1723306688,1723327305,1723374164,28800
4,Mumbai,28.00,haze,19.0761,72.8775,28.00,28.00,1009,78,1723306688,1723250893,1723297174,19800


In [16]:
cities_df = df[['city_ascii', 'lat', 'lng', 'admin_name','capital', 'population','country']]
new = pd.merge(cities_df, weather_df, on = ['lat', 'lng'], how = 'inner')

In [17]:
new.head()

,city_ascii,lat,lng,admin_name,capital,population,country,city,temperature,weather,temp_min,temp_max,sea_level,humidity,current_time,sunrise,sunset,timezone
0,Jakarta,-6.1750,106.8275,Jakarta,primary,33756000.0,Indonesia,Jakarta,28.34,few clouds,26.70,29.13,1011,69,1723306688,1723244492,1723287281,25200
1,Delhi,28.6100,77.2300,Delhi,admin,32226000.0,India,Delhi,29.10,mist,29.10,29.10,1001,89,1723306688,1723249063,1723296916,19800
2,Guangzhou,23.1300,113.2600,Guangdong,admin,26940000.0,China,Guangzhou,30.92,overcast clouds,30.92,30.92,1003,65,1723306688,1723327305,1723374164,28800
3,Mumbai,19.0761,72.8775,Mahārāshtra,admin,24973000.0,India,Mumbai,28.00,haze,28.00,28.00,1009,78,1723306688,1723250893,1723297174,19800
4,Sao Paulo,-23.5500,-46.6333,São Paulo,admin,23086000.0,Brazil,São Paulo,10.42,broken clouds,10.15,10.88,1021,87,1723306688,1723282541,1723322886,-10800


In [18]:
new.rename(columns = {'city_ascii' : 'city'}, inplace= True)
new['admin_name'].fillna('Unknown', inplace = True)
new['capital'].fillna('Unknown', inplace = True)

C:\Users\hamzumoh\AppData\Local\Temp\ipykernel_9120\190576525.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  new['admin_name'].fillna('Unknown', inplace = True)
C:\Users\hamzumoh\AppData\Local\Temp\ipykernel_9120\190576525.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

Fo

In [19]:
new['current_time'] = pd.to_datetime(new['current_time'], unit = 's')
new['sunrise'] = pd.to_datetime(new['sunrise'], unit = 's')
new['sunset'] = pd.to_datetime(new['sunset'], unit = 's')
new['day_length'] = np.ceil((new['sunset'] - new['sunrise']).dt.total_seconds() /3600)
new['Hot-cities'] = np.where(new['temperature'] > 30 , 'Y', 'N')

In [20]:
country_stats = new.groupby('country').agg({
    'temperature': 'mean',
    'population': 'sum',
    'humidity': 'mean'
}).reset_index()
country_stats['Country_AVG_popul_temp_humi'] = (country_stats['population'].astype(int).astype(str)) + ',' +round(country_stats['temperature'],2).astype(str) +','+ round(country_stats['humidity'],2).astype(str)

In [21]:
country_stats

,country,temperature,population,humidity,Country_AVG_popul_temp_humi
0,Algeria,34.115000,4983247.0,27.833333,"4983247,34.12,27.83"
1,Angola,24.226000,4506205.0,51.200000,"4506205,24.23,51.2"
2,Argentina,16.591667,22417111.0,36.333333,"22417111,16.59,36.33"
3,Australia,15.875000,2749499.0,82.000000,"2749499,15.88,82.0"
4,Austria,30.150000,3056616.0,46.600000,"3056616,30.15,46.6"
...,...,...,...,...,...
69,United Kingdom,22.832000,6150019.0,69.400000,"6150019,22.83,69.4"
70,United States,28.038333,57209960.0,58.833333,"57209960,28.04,58.83"
71,Uzbekistan,29.081667,5285040.0,35.666667,"5285040,29.08,35.67"
72,Venezuela,27.451667,8230390.0,63.333333,"8230390,27.45,63.33"


In [22]:
country_stats = country_stats[['country','Country_AVG_popul_temp_humi']]
final_df = pd.merge(new,country_stats, on='country', how = 'inner')

load

In [23]:
def load(data):
    os.environ['AWS_ACCESS_KEY_ID'] = 'Your AWS_ACCESS_KEY_ID'
    os.environ['AWS_SECRET_ACCESS_KEY'] = 'Your AWS_SECRET_ACCESS_KEY'
    csv_buffer = StringIO()
    data.to_csv(csv_buffer, index= False)
    s3 = boto3.client('s3', region_name= 'us-east-1')
    bucket_name = 'Your bucket name'
    file_name = 'Weather_Report.csv'
    s3.put_object(Bucket=bucket_name, Key=file_name, Body=csv_buffer.getvalue())

In [25]:
load(final_df)